In [23]:
#make sure to run pip install requirements.txt to install libraries
!pip install -r requirements.txt --quiet

# Data sourcing

The dataset we're using for that example is "The Movies Dataset".
It contains metadata on over 45,000 movies. 26 million ratings from over 270,000 users and can be downloaded here:
https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset

The dataset has a "CC0: Public Domain" license
https://creativecommons.org/publicdomain/zero/1.0/


## Download and extract the dataset

IMPORTANT: Note that you will need to authenticate in kaggle to download the file manually. 
https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset


Then, copy the file into the "dataset" folder.

In [24]:
import zipfile
import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

In [25]:
dataset_path = "../dataset"
zip_file = f'{dataset_path}/data.zip'

In [26]:
with zipfile.ZipFile(zip_file) as zf:
   zf.extractall(dataset_path)

In [27]:
#open files as dataframes
credits_df = pd.read_csv('../dataset/credits.csv')
keywords_df = pd.read_csv('../dataset/keywords.csv')
links_small_df = pd.read_csv('../dataset/links_small.csv')
links_df = pd.read_csv('../dataset/links.csv')
movies_metadata_df = pd.read_csv('../dataset/movies_metadata.csv')
ratings_small_df = pd.read_csv('../dataset/ratings_small.csv')
ratings_df = pd.read_csv('../dataset/ratings.csv')

/var/folders/mz/xzlrn3qx46g4g2v49620tgym0000gr/T/ipykernel_70187/3234850333.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata_df = pd.read_csv('../dataset/movies_metadata.csv')


In [28]:
#Popularity Columns (10) have mixed types, removing rows with popularity 
print(movies_metadata_df.shape)

movies_metadata_df["popularity"] = pd.to_numeric(movies_metadata_df["popularity"], errors="coerce")
movies_metadata_df = movies_metadata_df.dropna(subset=["popularity"])

print(movies_metadata_df.shape)

(45466, 24)
(45460, 24)


In [29]:
movies_metadata_df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


### Adding keywords to the main metadata dataframe

In [30]:
import ast
def add_keywords(id):
    result = []
    try:
        keywords = keywords_df.loc[keywords_df['id'] == int(id), "keywords"].iloc[0]
        list_of_dicts = ast.literal_eval(keywords)
        for _dict in list_of_dicts:
            result.append(_dict['name'])
        return ','.join(result)
    except Exception as e:
        print(f"Exception:{e}")
        return ""

#some rows have data quality issues, expect exceptions to be raised.
movies_metadata_df['keywords'] = movies_metadata_df['id'].apply(lambda x: add_keywords(x))

Exception:single positional indexer is out-of-bounds


### Adding Director column to the main metadata dataframe

In [31]:
def add_director(id):
    director = ''
    try:
        crew = credits_df.loc[credits_df['id'] == int(id), "crew"].iloc[0]
        list_of_dicts = ast.literal_eval(crew)
        for _dict in list_of_dicts:
            if _dict['job'] == 'Director':
                director = _dict['name']
                break
        return director
    except Exception as e:
        print(e)
        return director
    
#some rows have data quality issues, expect exceptions to be raised.
movies_metadata_df['director'] = movies_metadata_df['id'].apply(lambda x: add_director(x))

single positional indexer is out-of-bounds


### Adding Actors to the main metadata dataframe

In [32]:
def add_actors(id):
    actors = []
    try:
        cast = credits_df.loc[credits_df['id'] == int(id), "cast"].iloc[0]
        list_of_dicts = ast.literal_eval(cast)
        counter = 0
        for _dict in list_of_dicts:
            if _dict['order'] in [0,1,2]:
                actors.append(_dict['name'])
                counter += 1
                if counter == 3:
                    break
        return ",".join(actors)
    except Exception as e:
        print(e)
        return ""

#some rows have data quality issues, expect exceptions to be raised.
movies_metadata_df['actors'] = movies_metadata_df['id'].apply(lambda x: add_actors(x))

single positional indexer is out-of-bounds


### Round up popularity column

In [33]:
import math

#round value to first decimal
def update_popularity(num):
  try:
    return round(num, 1)
  except Exception as e:
    return 0

movies_metadata_df['popularity'] = movies_metadata_df['popularity'].apply(lambda x: update_popularity(x))

#casting as integer
movies_metadata_df["popularity"] = movies_metadata_df["popularity"].astype(float)

### Add popuplarity bins

In [34]:
movies_metadata_df["popularity_bins"] = pd.qcut(movies_metadata_df['popularity'], 5, labels=['Very Low', 'Low', 'Average', 'High', 'Very High'], duplicates='drop')

### Add vote_average  bins

In [35]:
movies_metadata_df["vote_average_bins"] = pd.qcut(movies_metadata_df['vote_average'], 5, labels=['Very Low', 'Low', 'Average', 'High', 'Very High'], duplicates='drop')

### Update genres column

In [36]:
def update_genre(json_genre):
    genres = []
    try:
        list_of_dicts = ast.literal_eval(json_genre)
        for _dict in list_of_dicts:
            genres.append(_dict['name'])
        return ",".join(genres)
    except Exception as e:
        print(e)
        return ""

movies_metadata_df['genres'] = movies_metadata_df['genres'].apply(lambda x: update_genre(x))

### Adding Year to the main metadata dataframe

In [37]:
def create_year(x):
    try:
        #expected format 1995-10-30
        return x.split('-')[0]
    except Exception as e:
        return ""

movies_metadata_df['year'] = movies_metadata_df['release_date'].apply(lambda x: create_year(x))

### Select which columns to export to csv

In [38]:
for col in movies_metadata_df.columns:
  null_count = movies_metadata_df[col].isnull().sum()
  print(f"Null count in {col}: {null_count}")

Null count in adult: 0
Null count in belongs_to_collection: 40970
Null count in budget: 0
Null count in genres: 0
Null count in homepage: 37682
Null count in id: 0
Null count in imdb_id: 17
Null count in original_language: 11
Null count in original_title: 0
Null count in overview: 954
Null count in popularity: 0
Null count in poster_path: 383
Null count in production_companies: 0
Null count in production_countries: 0
Null count in release_date: 84
Null count in revenue: 0
Null count in runtime: 257
Null count in spoken_languages: 0
Null count in status: 81
Null count in tagline: 25048
Null count in title: 0
Null count in video: 0
Null count in vote_average: 0
Null count in vote_count: 0
Null count in keywords: 0
Null count in director: 0
Null count in actors: 0
Null count in popularity_bins: 0
Null count in vote_average_bins: 0
Null count in year: 0


In [39]:
movies_metadata_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'keywords', 'director', 'actors',
       'popularity_bins', 'vote_average_bins', 'year'],
      dtype='object')

In [40]:
#renaming the id column to tmdb_id
movies_metadata_df.rename(columns={'id': 'tmdb_id'}, inplace=True)

#renaming the overview column to description
movies_metadata_df.rename(columns={'overview': 'description'}, inplace=True)

cols_selection = ['tmdb_id', 'original_language', 'original_title', 'description', 'genres', 'year', 'keywords', 'director', 'actors', 'popularity', 'popularity_bins',
                  'vote_average', 'vote_average_bins']

#full 45K dataset
to_export_df_full = movies_metadata_df[cols_selection]

In [41]:
#small version for workshop's purpose. we're selection the top 1000 most popular.
to_export_df_small = to_export_df_full.sort_values(by=['popularity'], ascending=[False])[:200]

In [42]:
to_export_df_small.head(10)

,tmdb_id,original_language,original_title,description,genres,year,keywords,director,actors,popularity,popularity_bins,vote_average,vote_average_bins
30700,211672,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...","Family,Animation,Adventure,Comedy",2015,"assistant,aftercreditsstinger,duringcreditssti...",Kyle Balda,"Sandra Bullock,Jon Hamm,Michael Keaton",547.5,Very High,6.4,High
33356,297762,en,Wonder Woman,An Amazon princess comes to the world of Man t...,"Action,Adventure,Fantasy",2017,"dc comics,hero,greek mythology,island,world wa...",Patty Jenkins,"Gal Gadot,Chris Pine,Robin Wright",294.3,Very High,7.2,Very High
42222,321612,en,Beauty and the Beast,A live-action adaptation of Disney's version o...,"Family,Fantasy,Romance",2017,"france,magic,castle,fairy tale,musical,curse,a...",Bill Condon,"Emma Watson,Dan Stevens,Luke Evans",287.3,Very High,6.8,High
43644,339403,en,Baby Driver,After being coerced into working for a crime b...,"Action,Crime",2017,"robbery,atlanta,music,crime boss,romance,tinni...",Edgar Wright,"Ansel Elgort,Lily James,Kevin Spacey",228.0,Very High,7.2,Very High
24455,177572,en,Big Hero 6,The special bond that develops between plus-si...,"Adventure,Family,Animation,Action,Comedy",2014,"brother brother relationship,hero,talent,reven...",Chris Williams,"Scott Adsit,Ryan Potter,Daniel Henney",213.8,Very High,7.8,Very High
26564,293660,en,Deadpool,Deadpool tells the origin story of former Spec...,"Action,Adventure,Comedy",2016,"anti hero,mercenary,marvel comic,superhero,bas...",Tim Miller,"Ryan Reynolds,Morena Baccarin,Ed Skrein",187.9,Very High,7.4,Very High
26566,283995,en,Guardians of the Galaxy Vol. 2,The Guardians must fight to keep their newfoun...,"Action,Adventure,Comedy,Science Fiction",2017,"sequel,superhero,based on comic,misfit,space,o...",James Gunn,"Chris Pratt,Zoe Saldana,Dave Bautista",185.3,Very High,7.6,Very High
14551,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...","Action,Adventure,Fantasy,Science Fiction",2009,"culture clash,future,space war,space colony,so...",James Cameron,"Sam Worthington,Zoe Saldana,Sigourney Weaver",185.1,Very High,7.2,Very High
24351,245891,en,John Wick,Ex-lunatic John Wick comes off his meds to tra...,"Action,Thriller",2014,"hitman,russian mafia,revenge,murder,gangster,d...",Chad Stahelski,"Keanu Reeves,Michael Nyqvist,Alfie Allen",183.9,Very High,7.0,High
23675,210577,en,Gone Girl,With his wife's disappearance having become th...,"Mystery,Thriller,Drama",2014,"based on novel,marriage crisis,disappearance,c...",David Fincher,"Ben Affleck,Rosamund Pike,Carrie Coon",154.8,Very High,7.9,Very High


In [43]:
#export
to_export_df_small.to_csv('../dataset/movies_metadata_small.csv', index=False)
to_export_df_full.to_csv('../dataset/movies_metadata_45K.csv', index=False)